# Consolidating data

This file consists of functions that consolidate our disparate datasets into one large dataset that is useful in training our model. 

The goal is to generate a file with 30 columns (this number should be variable), such that each column is a state in time. 

Ideally, this will be done with heirachical data, ie `p1` is the first point in time, and within `p1` you have an x component, y component, etc.

https://pandas.pydata.org/docs/user_guide/advanced.html

## Input data format

It is assumed that the input data with have the columns: `[timestamp,tx,ty,tz,qx,qy,qz,qw]`

## Extracting the data we want

This function will create velocity and acceleration columns.

In [ ]:
import numpy as np
import pandas as pd

def extract_features(raw: pd.DataFrame, dropna: bool = False) -> None:
    raw['vx'] = raw['tx'].diff() / raw['timestamp'].diff()
    raw['vy'] = raw['ty'].diff() / raw['timestamp'].diff()
    raw['vz'] = raw['tz'].diff() / raw['timestamp'].diff()

    raw['ax'] = raw['vx'].diff() / raw['timestamp'].diff()
    raw['ay'] = raw['vy'].diff() / raw['timestamp'].diff()
    raw['az'] = raw['vz'].diff() / raw['timestamp'].diff()

    if dropna: raw.dropna(inplace=True)

In [ ]:
# test the above functions

df = pd.read_csv("../data/fpv_uzh/indoor_forward_3_davis_with_gt.txt")

extract_features(df, dropna=True)

print(df.head())
print(df['timestamp'])

## Slicing the data

Now, we want rows of data that represent a specific range of time. In this case, we want 30 points for each new row.

In [ ]:
def generate_slices(data: pd.DataFrame, n: int) -> pd.DataFrame:
    # each row in the original data is a "point". Each row in the output 
    # is a list of points of size n. 
    indices = [[f"p{i}" for i in range(n)], data.columns]
    m_index = pd.MultiIndex.from_product(indices, names=["points", "data"])
    slices = []
    for i in range(len(data) - n):
        flattened = pd.DataFrame([data[i:i+n].to_numpy().flatten()], index=m_index, columns=data.columns)
        print('flattened', flattened.columns)
        slices.append(flattened)
        
        # redo this later
        # for k in range(n):
        #     out[f"p{k}"] = slice[k]
        
    return pd.concat(slices, ignore_index=False)

# test the above function

slices = generate_slices(df, 4)
print(slices.columns)

## Consolidate all our original data

Now, we want to consolidate our data from all the other sources.

In [ ]:
import os

fpv_data = "../data/fpv_uzh"
random_traj_data = "../data/random_trajectory_100ms"
output_path = "../data/output"
if not os.path.exists(output_path):
    os.makedirs(output_path)

n = 30 # we want 30 points per row
slices = []

# consolidate the fpv data
for filename in filter(lambda p: p.endswith("txt"), os.listdir(fpv_data)):
    filepath = os.path.join(fpv_data, filename)
    df = pd.read_csv(filepath)
    
    extract_features(df)
    # the columns should be handled in the generate slices funciton
    slices.append(generate_slices(df, n))
    
# consolidate the synthetic data
for filename in filter(lambda p: p.endswith("txt"), os.listdir(random_traj_data)):
    filepath = os.path.join(random_traj_data, filename)
    df = pd.read_csv(filepath)
    
    extract_features(df)
    # the columns should be handled in the generate slices funciton
    slices.append(generate_slices(df, n))

consolidated = pd.concat(slices, ignore_index=False)
consolidated.to_csv(os.path.join(output_path, "consolidated.csv"))